In [60]:
import httpx
import polars as pl

In [61]:
INE_BASE_URL = "https://servicios.ine.es/wstempus/js/ES"

client = httpx.Client(
    base_url=INE_BASE_URL,
    limits=httpx.Limits(max_keepalive_connections=20),
    transport=httpx.HTTPTransport(retries=3),
)

In [62]:
def ine_request(client: httpx.Client, endpoint, paginate=True):
    page = 1
    data = []

    while True:
        params = {"det": 10}

        if paginate:
            params["page"] = page

        response = client.get(
            f"/{endpoint}", params=params, follow_redirects=True
        ).json()

        if not response:
            break

        data.extend(response)

        if len(response) < 500 or not paginate:
            break

        page += 1

    return data

In [63]:
# https://servicios.ine.es/wstempus/js/ES/OPERACIONES_DISPONIBLES

operaciones_disponibles = pl.DataFrame(ine_request(client, "OPERACIONES_DISPONIBLES"))
print(operaciones_disponibles.shape)
operaciones_disponibles.sample(5)

(108, 6)


Id,Cod_IOE,Nombre,Codigo,Referencia,Url
i64,str,str,str,list[struct[3]],str
27,"""30051""","""Índices de Precios Industriale…","""IPRI""","[{22,""Índices de precios industriales"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736147699&idp=1254735576715""}]",null
30,"""""","""Encuesta de coyuntura de comer…","""CCM""",null,null
49,"""""","""Distribución de Apellidos""","""APEL""",null,"""https://www.ine.es/dyngs/INEba…"
465,"""30240""","""Comercio Internacional de Serv…","""STEC""",null,null
104,"""30062""","""Índices de Precios de Material…","""IMM""","[{30,""Índices de precios de materiales e índices nacionales de la mano de obra"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736154972&idp=1254735576757""}]",null


In [64]:
# https://servicios.ine.es/wstempus/js/ES/VARIABLES?det=10

variables = pl.DataFrame(ine_request(client, "VARIABLES"))
variables.sample(5)

Id,Nombre,Codigo
i64,str,str
178,"""Sectores económicos del empleo…",""""""
600,"""Conurbación""","""KERNEL"""
771,"""Relacion entre lugar de nacimi…",""""""
807,"""Códigos de gasto con cantidad …",""""""
170,"""Motivo de no buscar empleo""",""""""


In [65]:
t = []

for row in operaciones_disponibles.rows(named=True):
    tablas_operacion_url = f"TABLAS_OPERACION/{row["Id"]}"
    tablas_operacion = ine_request(client, tablas_operacion_url)

    t.extend(tablas_operacion)

In [67]:
import os

os.makedirs("dataset", exist_ok=True)

tablas = pl.json_normalize(t)
print(tablas.shape)
tablas.write_parquet("dataset/tablas.parquet", compression="zstd")
tablas.sample(5)

(4934, 50)


Id,Nombre,Codigo,Anyo_Periodo_ini,FechaRef_fin,Ultima_Modificacion,Periodicidad.Id,Periodicidad.Nombre,Periodicidad.Codigo,Publicacion.Id,Publicacion.Nombre,Publicacion.Periodicidad.Id,Publicacion.Periodicidad.Nombre,Publicacion.Periodicidad.Codigo,Publicacion.Operacion,Publicacion.PubFechaAct.Id,Publicacion.PubFechaAct.Nombre,Publicacion.PubFechaAct.Fecha,Publicacion.PubFechaAct.Periodo.Id,Publicacion.PubFechaAct.Periodo.Valor,Publicacion.PubFechaAct.Periodo.Periodicidad.Id,Publicacion.PubFechaAct.Periodo.Periodicidad.Nombre,Publicacion.PubFechaAct.Periodo.Periodicidad.Codigo,Publicacion.PubFechaAct.Periodo.Dia_inicio,Publicacion.PubFechaAct.Periodo.Mes_inicio,Publicacion.PubFechaAct.Periodo.Codigo,Publicacion.PubFechaAct.Periodo.Nombre,Publicacion.PubFechaAct.Periodo.Nombre_largo,Publicacion.PubFechaAct.Anyo,Periodo_ini.Id,Periodo_ini.Valor,Periodo_ini.Periodicidad.Id,Periodo_ini.Periodicidad.Nombre,Periodo_ini.Periodicidad.Codigo,Periodo_ini.Dia_inicio,Periodo_ini.Mes_inicio,Periodo_ini.Codigo,Periodo_ini.Nombre,Periodo_ini.Nombre_largo,Anyo_Periodo_fin,Periodo_fin.Id,Periodo_fin.Valor,Periodo_fin.Periodicidad.Id,Periodo_fin.Periodicidad.Nombre,Periodo_fin.Periodicidad.Codigo,Periodo_fin.Dia_inicio,Periodo_fin.Mes_inicio,Periodo_fin.Codigo,Periodo_fin.Nombre,Periodo_fin.Nombre_largo
i64,str,str,str,str,i64,i64,str,str,i64,str,i64,str,str,list[struct[5]],i64,str,i64,i64,i64,i64,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,str,str,str,str,str,str,i64,i64,i64,str,str,str,str,str,str,str
2861,"""Barcelona: Población por munic…","""PROV-MUN""","""1996""","""null""",1703584800000,12,"""Anual""","""A""",29,"""Cifras oficiales de población …",12,"""Anual""","""A""","[{22,""30245"",""Cifras Oficiales de Población de los Municipios Españoles: Revisión del Padrón Municipal"",""DPOP"",[{119,""Cifras oficiales de población de los municipios españoles: Revisión del Padrón Municipal"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177011&idp=1254734710990""}]}]",10681,"""Cifras oficiales de población …",1703584800000,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""",2023,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""",null,null,null,null,null,null,null,null,null,null,null
2255,"""Índices nacionales: general y …","""2005_NAC_PCTES""","""2003""",null,1388390400000,1,"""Mensual""","""M""",5,"""Índices de Comercio al por men…",1,"""Mensual""","""M""","[{32,""30103"",""Índices de Comercio al por Menor"",""ICM"",[{60,""Índices de comercio al por menor"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176900&idp=1254735576799""}]}]",10511,"""Índices de Comercio al por men…",1727334000000,8,8,1,"""Mensual""","""M""","""1""","""8""","""08""","""Agosto""","""Agosto""",2024,1,1,1,"""Mensual""","""M""","""1""","""1""","""01""","""Enero""","""Enero""","""2008""",12,12,1,"""Mensual""","""M""","""1""","""12""","""12""","""Diciembre""","""Diciembre"""
66068,"""Ocupados por situación profesi…","""NAC""","""2006""",null,1406624400000,12,"""Anual""","""A""",341,"""Encuesta de Población Activa. …",12,"""Anual""","""A""","[{293,""30308"",""Encuesta de Población Activa (EPA)"",""EPA"",[{138,""Encuesta de población activa"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176918&idp=1254735976595""}]}]",10872,"""Encuesta de Población Activa. …",1718965380000,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""",2023,28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año""","""2010""",28,1,12,"""Anual""","""A""","""1""","""1""","""01""","""A""","""Año"""
28401,"""Personas condenadas en asuntos…","""NAC-CCAA""","""2015""","""null""",1715936400000,12,"""Anual""","""A""",312,"""Estadística de Violencia Domés…",12,"""Anual""","""A""","[{215,""30468"",""Estadística de Violencia Doméstica y Violencia de Género"",""VGD"",[{171,""Estadística de violencia doméstica y violencia de género"",""/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=12547361768

In [68]:
def generate_readme_content(data):
    return f"""# {data['Nombre']}

- **ID:** {data['Id']}
- **Código:** {data['Codigo']}
- **Año de Inicio del Periodo:** {data['Anyo_Periodo_ini']}
- **Fecha de Referencia Final:** {data['FechaRef_fin']}
- **Última Modificación:** {data['Ultima_Modificacion']}
- **Periodicidad:** {data['Periodicidad.Nombre']} ({data['Periodicidad.Codigo']})

## Publicación

- **ID:** {data['Publicacion.Id']}
- **Nombre:** {data['Publicacion.Nombre']}
- **Periodicidad:** {data['Publicacion.Periodicidad.Nombre']} ({data['Publicacion.Periodicidad.Codigo']})

### Operación

- **ID:** {data['Publicacion.Operacion'][0]['Id']}
- **Código IOE:** {data['Publicacion.Operacion'][0]['Cod_IOE']}
- **Nombre:** {data['Publicacion.Operacion'][0]['Nombre']}
- **Referencia:** {data['Publicacion.Operacion'][0]['Referencia']}

### Última Publicación

- **ID:** {data['Publicacion.PubFechaAct.Id']}
- **Nombre:** {data['Publicacion.PubFechaAct.Nombre']}
- **Fecha:** {data['Publicacion.PubFechaAct.Fecha']}
- **Periodo ID:** {data['Publicacion.PubFechaAct.Periodo.Id']}
- **Periodo Valor:** {data['Publicacion.PubFechaAct.Periodo.Valor']}
- **Periodo Periodicidad:** {data['Publicacion.PubFechaAct.Periodo.Periodicidad.Nombre']} ({data['Publicacion.PubFechaAct.Periodo.Periodicidad.Codigo']})
- **Periodo Día de Inicio:** {data['Publicacion.PubFechaAct.Periodo.Dia_inicio']}
- **Periodo Mes de Inicio:** {data['Publicacion.PubFechaAct.Periodo.Mes_inicio']}
- **Periodo Código:** {data['Publicacion.PubFechaAct.Periodo.Codigo']}
- **Periodo Nombre:** {data['Publicacion.PubFechaAct.Periodo.Nombre']}
- **Periodo Nombre Largo:** {data['Publicacion.PubFechaAct.Periodo.Nombre_largo']}
- **Año:** {data['Publicacion.PubFechaAct.Anyo']}

## Periodo Inicial

- **ID:** {data['Periodo_ini.Id']}
- **Valor:** {data['Periodo_ini.Valor']}
- **Periodicidad:** {data['Periodo_ini.Periodicidad.Nombre']} ({data['Periodo_ini.Periodicidad.Codigo']})
- **Día de Inicio:** {data['Periodo_ini.Dia_inicio']}
- **Mes de Inicio:** {data['Periodo_ini.Mes_inicio']}
- **Código:** {data['Periodo_ini.Codigo']}
- **Nombre:** {data['Periodo_ini.Nombre']}
- **Nombre Largo:** {data['Periodo_ini.Nombre_largo']}

## Periodo Final

- **Año de Fin del Periodo:** {data['Anyo_Periodo_fin']}
- **ID:** {data['Periodo_fin.Id']}
- **Valor:** {data['Periodo_fin.Valor']}
- **Periodicidad:** {data['Periodo_fin.Periodicidad.Nombre']} ({data['Periodo_fin.Periodicidad.Codigo']})
- **Día de Inicio:** {data['Periodo_fin.Dia_inicio']}
- **Mes de Inicio:** {data['Periodo_fin.Mes_inicio']}
- **Código:** {data['Periodo_fin.Codigo']}
- **Nombre:** {data['Periodo_fin.Nombre']}
- **Nombre Largo:** {data['Periodo_fin.Nombre_largo']}

## JSON

```json
{data}
```
"""

In [69]:
def get_series_tabla_url(tabla_id):
    return (
        f"https://servicios.ine.es/wstempus/jsCache/ES/SERIES_TABLA/{tabla_id}?det=10"
    )


def get_tablas_download_url(tabla_id):
    return f"https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/{tabla_id}.csv"


tablas = tablas.with_columns(
    pl.col("Id")
    .map_elements(get_series_tabla_url, return_dtype=pl.String)
    .alias("series_tabla_url"),
    pl.col("Id")
    .map_elements(get_tablas_download_url, return_dtype=pl.String)
    .alias("tablas_download_url"),
)

In [70]:
tablas = tablas.sample(10)

In [71]:
with open("series.input.spec", "w") as f:
    for t in tablas.rows(named=True):
        f.write(f"{t['series_tabla_url']}\n")
        f.write("\tout=metadata.json\n")
        f.write(f"\tdir=dataset/tablas/{t['Id']}\n\n")

with open("tablas.input.spec", "w") as f:
    for t in tablas.rows(named=True):
        f.write(f"{t['tablas_download_url']}\n")
        f.write(f"\tout={t['Id']}.csv\n")
        f.write(f"\tdir=dataset/tablas/{t['Id']}\n\n")

In [72]:
import subprocess

series_result = subprocess.run(
    [
        "aria2c",
        "-i",
        "series.input.spec",
        "-j",
        "50",
        "-x",
        "16",
        "-s",
        "8",
        "-c",
        "--file-allocation=none",
        "--console-log-level=warn",
    ],
    capture_output=True,
    text=True,
)

In [75]:
print(series_result.stdout)

[DL:5.5MiB][#266a5e 1.2MiB/0B][#c590df 0B/0B][#a316be 0B/0B][#82fbf4 0B/0B]
[DL:2.6MiB][#c590df 0B/0B][#a316be 0B/0B][#82fbf4 0B/0B]
[DL:1.7MiB][#c590df 0B/0B][#a316be 0B/0B][#82fbf4 0B/0B]
[DL:2.5MiB][#c590df 1.5MiB/0B][#a316be 725KiB/0B][#82fbf4 2.8MiB/0B]
[DL:4.3MiB][#c590df 5.5MiB/0B][#a316be 4.7MiB/0B]
[DL:4.9MiB][#c590df 9.5MiB/0B][#a316be 8.7MiB/0B]
[#c590df 13MiB/0B CN:1 DL:3.9MiB]
[#c590df 17MiB/0B CN:1 DL:3.9MiB]
[#c590df 21MiB/0B CN:1 DL:3.9MiB]

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
b9c635|OK  |   3.6MiB/s|dataset/tablas/70173/metadata.json
d624d4|OK  |    26KiB/s|dataset/tablas/20170/metadata.json
fc1a17|OK  |   285KiB/s|dataset/tablas/49127/metadata.json
f83b0d|OK  |   196KiB/s|dataset/tablas/20509/metadata.json
88d0be|OK  |   4.0MiB/s|dataset/tablas/10634/metadata.json
a3fd1f|OK  |   189KiB/s|dataset/tablas/65934/metadata.json
266a5e|OK  |   1.3MiB/s|dataset/tablas/70694/metadata

In [76]:
tablas_result = subprocess.run(
    [
        "aria2c",
        "-i",
        "tablas.input.spec",
        "-j",
        "50",
        "-x",
        "16",
        "-s",
        "8",
        "-c",
        "--file-allocation=none",
        "--console-log-level=warn",
    ],
    capture_output=True,
    text=True,
)

In [77]:
print(tablas_result.stdout)

[DL:8.6MiB][#f3fb6c 360KiB/0B][#ad45d8 1.5MiB/0B][#81b592 436KiB/0B][#d72fb3 0.9MiB/0B]

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
b47fac|OK  |    81KiB/s|dataset/tablas/20509/20509.csv
eee19f|OK  |   123KiB/s|dataset/tablas/70694/70694.csv
ede36a|OK  |    10KiB/s|dataset/tablas/20170/20170.csv
45f220|OK  |   221KiB/s|dataset/tablas/65934/65934.csv
ff98a4|OK  |    84KiB/s|dataset/tablas/49127/49127.csv
163190|OK  |   1.8MiB/s|dataset/tablas/25039/25039.csv
d72fb3|OK  |   1.4MiB/s|dataset/tablas/10634/10634.csv
f3fb6c|OK  |   767KiB/s|dataset/tablas/70173/70173.csv
ad45d8|OK  |   3.2MiB/s|dataset/tablas/32888/32888.csv
81b592|OK  |   1.9MiB/s|dataset/tablas/31232/31232.csv

Status Legend:
(OK):download completed.



In [78]:
import glob

from tqdm import tqdm

csv_files = glob.glob("dataset/tablas/*/*.csv")

for file in tqdm(csv_files):
    filename = file.split(".")[-2].split("/")[-1]

    table_metadata = tablas.filter(pl.col("Id") == int(filename)).to_struct()[0]

    with open(f"dataset/tablas/{filename}/README.md", "w") as f:
        f.write(generate_readme_content(table_metadata))

    (
        pl.scan_csv(
            file,
            separator=";",
            ignore_errors=True,
            truncate_ragged_lines=True,
        ).sink_parquet(
            f"dataset/tablas/{filename}/{filename}.parquet",
            compression="zstd",
            row_group_size=1024**2,
            type_coercion=True,
        )
    )


100%|██████████| 10/10 [00:00<00:00, 85.09it/s]


In [79]:
import os

for file in csv_files:
    os.remove(file)

In [39]:
from huggingface_hub import HfApi

In [40]:
api = HfApi(token=os.getenv("HUGGINGFACE_TOKEN"))

In [42]:
api.upload_large_folder(
    folder_path="dataset", repo_id="davidgasquez/ine", repo_type="dataset"
)